## Initial and boundary conditions


In [ ]:
# Define Dirichlet boundary conditions
if any ([system == 1, system == 2]): 
    # Define Dirichlet boundary conditions for smooth flow
    if IBVP == 1:
        variable1_dirichlet = Constant (ref[0])
        variable2_dirichlet = Constant (ref[1])
        variable3_dirichlet = Constant (ref[2])
        variable4_dirichlet = Constant (ref[3])

    # Define Dirichlet boundary conditions for perturbed flow
    elif IBVP == 2:
        # ang_frequency = real_part[0][1][1]
        # eigenvector = waves_effect*np.asarray([ 0.00291852-2.81805376e-04j,  0.00293087-5.94800113e-05j, -0.05152862+1.02129577e-04j, -0.01828003-4.75631527e-02j])

        # Initial time
        tbc = 0
        
        # Reference
        variable1_dirichlet = Constant (ref[0])
        variable2_dirichlet = Constant (ref[1])
        variable3_dirichlet = Constant (ref[2])
        variable4_dirichlet = Constant (ref[3])

        # Amplitude
        amplitude1 = (vect2[0]).real
        amplitude2 = (vect2[1]).real
        amplitude3 = (vect2[2]).real
        amplitude4 = (vect2[3]).real
        
        # Anglular frequency
        

        # Perturbation wave
        variable1_dirichlet = Expression ('variable1_dirichlet + amplitude1*sin (ang_frequency*tbc - wavenumber*x[0])', 
                                            degree = deg1, 
                                            variable1_dirichlet = variable1_dirichlet, 
                                            amplitude1 = amplitude1, 
                                            wavenumber  = wavenumber_fourier, 
                                            ang_frequency = ang_frequency, 
                                            tbc = tbc, 
                                            domain = mesh)
        variable2_dirichlet = Expression ('variable2_dirichlet + amplitude2*sin (ang_frequency*tbc - wavenumber*x[0])', 
                                            degree = deg2, 
                                            variable2_dirichlet = variable2_dirichlet, 
                                            amplitude2 = amplitude2, 
                                            wavenumber  = wavenumber_fourier, 
                                            ang_frequency = ang_frequency, 
                                            tbc = tbc, 
                                            domain = mesh)
        variable3_dirichlet = Expression ('variable3_dirichlet + amplitude3*sin (ang_frequency*tbc - wavenumber*x[0])', 
                                            degree = deg3, 
                                            variable3_dirichlet = variable3_dirichlet, 
                                            amplitude3 = amplitude3, 
                                            wavenumber  = wavenumber_fourier, 
                                            ang_frequency = ang_frequency, 
                                            tbc = tbc, 
                                            domain = mesh)
        variable4_dirichlet = Expression ('variable4_dirichlet + amplitude4*sin (ang_frequency*tbc - wavenumber*x[0])', 
                                            degree = deg4, 
                                            variable4_dirichlet = variable4_dirichlet, 
                                            amplitude4 = amplitude4, 
                                            wavenumber  = wavenumber_fourier, 
                                            ang_frequency = ang_frequency, 
                                            tbc = tbc, 
                                            domain = mesh)
elif system == 3:
    if IBVP == 1:
        # Define Dirichlet boundary conditions for smooth flow
        variable1_dirichlet = Constant (ref[0]/rho_l)
        variable2_dirichlet = Constant (ref[2]/ref[0])
        variable3_dirichlet = Constant (ref[3]/ref[1])
        variable4_dirichlet = Constant (ref[1]/((1 - ref[0]/rho_l)*p_factor))

# Define vector of boundary conditions
if any ([elementspace == 1, elementspace == 2, elementspace == 3]):
    # Dirichlet boundary conditions
    bc1 = DirichletBC (V.sub (0), variable1_dirichlet, Inflow)#, "geometric") # "geometric" "pointwise"
    bc2 = DirichletBC (V.sub (1), variable2_dirichlet, Inflow)#, "geometric")
    bc3 = DirichletBC (V.sub (2), variable3_dirichlet, Inflow)#, "geometric")
    bc4 = DirichletBC (V.sub (3), variable4_dirichlet, Outflow)#, "geometric")
    bcs = [bc1, bc2, bc3, bc4]
elif elementspace == 4:
    # Define ds for facets
    # DOLFIN predefines the “measures” dx, ds and dS representing integration over cells, exterior facets (that is, facets on the boundary) and interior facets, respectively. These measures can take an additional integer argument. In fact, dx defaults to dx(0), ds defaults to ds(0), and dS defaults to dS(0). Integration over subregions can be specified by measures with different integer labels as arguments.
    # Define outer surface measure aware of Dirichlet boundaries
    ds = Measure ('ds', domain = mesh, subdomain_data = boundaries)
    # Dirichlet boundary conditions
    bc1 = DirichletBC (V.sub (0), variable1_dirichlet, boundaries, 1)#, "geometric") # "geometric"<<<<
    bc2 = DirichletBC (V.sub (1), variable2_dirichlet, boundaries, 1)#, "geometric")
    bc3 = DirichletBC (V.sub (2), variable3_dirichlet, boundaries, 1)#, "geometric")
    bc4 = DirichletBC (V.sub (3), variable4_dirichlet, boundaries, 2)#, "geometric")
    bcs = [bc1, bc2, bc3, bc4]

# Initial conditions
if IBVP == 1:
    variable_init = InitialConditions_SS ()
elif IBVP == 2:
    variable_init = InitialConditions_SS_perturbed ()

# interpolate vector of initial conditions
variable_ic.interpolate (variable_init)

# interpolate vector of initial conditions (transient simulations)
if any ([time_method == 1, time_method == 3]):
    variable_n.interpolate (variable_init)
elif time_method == 2:
    variable_past.interpolate (variable_init)
    
# Show initial conditions
if show_data == 1:
    if any ([system == 1, system == 2]):
        plt.figure (1)
        fig, ax = plt.subplots ()
        # plt.ylim (0, 1)
        plt.xlim (0, L)
        plt.grid (True, which = "both")
        ax.set_xlabel (r'L [m]')
        ax.set_ylabel (r'$\alpha_l$ [-]')
        ax.xaxis.set_tick_params (which = 'major', direction = 'in', top = 'on')
        ax.xaxis.set_tick_params (which = 'minor', direction = 'in', top = 'on')
        ax.yaxis.set_tick_params (which = 'major', direction = 'in', right = 'on')
        ax.yaxis.set_tick_params (which = 'minor', direction = 'in', right = 'on')
        plot (variable1_ic, color = 'k')

        # Save plot
        fig.set_size_inches (mapsize)
        plt.savefig('figures/initial_conditions/fig1.pdf',
                    optimize = True,
                    transparent = True,  
                    dpi = dpi_elsevier)

        plt.figure (2)
        fig, ax = plt.subplots ()
        plt.xlim (0, L)
        plt.grid (True, which = "both")
        ax.set_xlabel (r'L [m]')
        ax.set_ylabel (r'$u_l$ [m/s]')
        ax.xaxis.set_tick_params (which = 'major', direction = 'in', top = 'on')
        ax.xaxis.set_tick_params (which = 'minor', direction = 'in', top = 'on')
        ax.yaxis.set_tick_params (which = 'major', direction = 'in', right = 'on')
        ax.yaxis.set_tick_params (which = 'minor', direction = 'in', right = 'on')
        plot (variable2_ic, color = 'k')

        # Save plot 
        fig.set_size_inches (mapsize)
        plt.savefig('figures/initial_conditions/fig2.pdf',
                    optimize = True,
                    transparent = True,  
                    dpi = dpi_elsevier)

        plt.figure (3)
        fig, ax = plt.subplots ()
        plt.xlim (0, L)
        plt.grid (True, which = "both")
        ax.set_xlabel (r'L [m]')
        ax.set_ylabel (r'$u_g$ [m/s]')
        ax.xaxis.set_tick_params (which = 'major', direction = 'in', top = 'on')
        ax.xaxis.set_tick_params (which = 'minor', direction = 'in', top = 'on')
        ax.yaxis.set_tick_params (which = 'major', direction = 'in', right = 'on')
        ax.yaxis.set_tick_params (which = 'minor', direction = 'in', right = 'on')
        plot (variable3_ic, color = 'k')

        # Save plot
        fig.set_size_inches (mapsize)
        plt.savefig('figures/initial_conditions/fig3.pdf',
                    optimize = True,
                    transparent = True,  
                    dpi = dpi_elsevier)

        plt.figure (4)
        fig, ax = plt.subplots ()
        plt.xlim (0, L)
        plt.grid (True, which = "both")
        ax.set_xlabel (r'L [m]')
        ax.set_ylabel (r'$p_i$ [Pa]')
        ax.xaxis.set_tick_params (which = 'major', direction = 'in', top = 'on')
        ax.xaxis.set_tick_params (which = 'minor', direction = 'in', top = 'on')
        ax.yaxis.set_tick_params (which = 'major', direction = 'in', right = 'on')
        ax.yaxis.set_tick_params (which = 'minor', direction = 'in', right = 'on')
        plot (variable4_ic, color = 'k')

        # Save plot
        fig.set_size_inches (mapsize)
        plt.savefig('figures/initial_conditions/fig4.pdf',
                    optimize = True,
                    transparent = True,  
                    dpi = dpi_elsevier)